In [3]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BaseEstimatorV2 as Estimator
from qiskit_algorithms.minimum_eigensolvers import QAOA
from qiskit_algorithms.optimizers import COBYLA

# ---------- 1) Build the multi-component Ising from your J, K, eps ----------

def build_multi_component_Ising(J, K=2, eps=0.2, scale_to=1.0):
    """
    Return (h, J_ising), variable order: i = k*N + n  (k in [0..K-1], n in [0..N-1])
    We MAXIMIZE   M = (K+eps) ⊗_k (r_k^T J r_k)  - eps * sum_{k≠l} r_k^T J r_l
    QAOA MINIMIZES energy, so we use H = -M (i.e., cost Hamiltonian encodes minimization of -M).
    """
    N = J.shape[0]
    J = 0.5*(J + J.T)             # ensure symmetry
    I_K = np.eye(K)
    onesK = np.ones((K, K))

    # Target maximize matrix M (same idea as your annealing build)
    M = (K + eps) * np.kron(I_K, J) - eps * np.kron(onesK - I_K, J)

    # Convert to "minimize" cost: H = -M
    H = -M

    # Optional scaling (keep couplers in a nice numeric range)
    max_abs = np.max(np.abs(H))
    if max_abs > 0:
        H = H * (scale_to / max_abs)

    # Extract linear h (we'll keep it zero; diagonals contribute only constants for ±1 spins)
    NK = N * K
    h = np.zeros(NK)

    # Extract quadratic dictionary (upper-triangular)
    Jij = {}
    for i in range(NK):
        for j in range(i+1, NK):
            val = H[i, j]
            if abs(val) > 1e-12:
                Jij[(i, j)] = float(val)

    return h, Jij, N, K

# ---------- 2) Convert (h, J) to a Pauli-Z cost operator ----------

def ising_to_pauli_op(h, Jij, n_qubits):
    """
    Build SparsePauliOp for H = sum_i h_i Z_i + sum_{i<j} J_ij Z_i Z_j (constants dropped).
    """
    paulis = []
    coeffs = []

    # linear Z terms
    for i in range(n_qubits):
        if abs(h[i]) > 1e-12:
            z_str = ['I'] * n_qubits
            z_str[i] = 'Z'
            paulis.append(''.join(reversed(z_str)))  # Qiskit little-endian convention
            coeffs.append(h[i])

    # quadratic ZZ terms
    for (i, j), w in Jij.items():
        z_str = ['I'] * n_qubits
        z_str[i] = 'Z'
        z_str[j] = 'Z'
        paulis.append(''.join(reversed(z_str)))
        coeffs.append(w)

    if not paulis:
        # No terms -> zero operator
        return SparsePauliOp.from_list([('I'*n_qubits, 0.0)])

    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))


In [7]:

def run_qaoa_on_Ising(h, Jij, n_qubits, p=2, maxiter=200, initial_params=None, seed=1):
    """
    Returns: (best_bitstring, best_energy, qaoa_result)
    """
    cost_op = ising_to_pauli_op(h, Jij, n_qubits)

    optimizer = COBYLA(maxiter=maxiter)
    estimator = Estimator()  # statevector sim by default; swap to hardware Estimator for real QPU

    qaoa = QAOA(estimator=estimator, optimizer=optimizer, reps=p, initial_point=initial_params, seed=seed)
    result = qaoa.compute_minimum_eigenvalue(operator=cost_op)

    # Sample the optimized circuit to get bitstrings (optional but useful)
    # Here, we synthesize the optimal parameters into a circuit and sample classically.
    qc = qaoa.ansatz
    params = result.optimal_point
    bound = qc.bind_parameters(params)

    # Use a quick classical sampler (statevector probabilities) to extract the best bitstring:
    # For large n, switch to Aer Sampler to get bitstrings.
    from qiskit.quantum_info import Statevector
    sv = Statevector.from_instruction(bound)
    probs = sv.probabilities_dict()

    # cost evaluation per bitstring (pick argmin energy)
    def bit_energy(bitstr):
        # compute <bit|H|bit> classically
        e = 0.0
        # linear
        for i in range(n_qubits):
            if abs(h[i]) > 1e-12:
                zi = 1 if bitstr[-1 - i] == '0' else -1  # |0>→+1 eigenvalue of Z; |1>→-1
                e += h[i] * zi
        # quadratic
        for (i, j), w in Jij.items():
            zi = 1 if bitstr[-1 - i] == '0' else -1
            zj = 1 if bitstr[-1 - j] == '0' else -1
            e += w * zi * zj
        return e

    best_bs, best_E = None, None
    for bs, p in probs.items():
        if p < 1e-12:
            continue
        e = bit_energy(bs)
        if (best_E is None) or (e < best_E):
            best_E, best_bs = e, bs

    return best_bs, best_E, result

# ---------- 4) Helper to map bitstring → spins → B_opt (N×K) ----------

def bitstring_to_Bopt(bitstr, N, K):
    """
    bitstr is little-endian in Qiskit dictionary keys, but we read it consistently above.
    Map |0>→+1, |1>→-1 and reshape to (N,K) with variable index i = k*N + n.
    """
    spins = []
    for i in range(N*K):
        # i-th variable corresponds to bit at position i from the *right* in our energy eval
        b = bitstr[-1 - i]
        spins.append(1 if b == '0' else -1)
    B = np.array(spins, dtype=int).reshape(K, N).T
    return B


rng = np.random.default_rng(0)
N, K = 6, 2
A = rng.normal(size=(N, N))
J = A @ A.T
J /= np.max(np.abs(J))

eps = 0.2
h, Jij, N_, K_ = build_multi_component_Ising(J, K=K, eps=eps, scale_to=1.0)
n_qubits = N_ * K_

best_bs, best_E, qres = run_qaoa_on_Ising(h, Jij, n_qubits, p=2, maxiter=150, seed=3)
print("Best energy:", best_E)
print("Best bitstring:", best_bs)

B_opt = bitstring_to_Bopt(best_bs, N_, K_)
print("B_opt shape:", B_opt.shape, "unique:", np.unique(B_opt))
# Now B_opt ∈ {±1}^{N×K} — use it in your post-step U = Q(X B_opt) if needed.

TypeError: Can't instantiate abstract class BaseEstimatorV2 without an implementation for abstract method 'run'